# Data Reshaping + Tidy Data

In [1]:
import pandas as pd
import numpy as np
import env

DB_URL = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

## Data Reshaping

- **long** data has many rows and few columns
- **wide** data has many columns
- a **melt** takes the data from wide to long
- a **spread**, or **pivot** takes the data from long to wide
- a **transpose** rotates the dataframe 90 degrees

### Melt Example

In [2]:
np.random.seed(123)

# simple data for demonstration
df = pd.DataFrame({
    'a': np.random.randint(1, 11, 3),
    'b': np.random.randint(1, 11, 3),
    'c': np.random.randint(1, 11, 3),
    'x': np.random.randint(1, 11, 3),
    'y': np.random.randint(1, 11, 3),
    'z': np.random.randint(1, 11, 3),    
})
df

,a,b,c,x,y,z
0,3,2,7,2,1,5
1,3,4,2,10,10,1
2,7,10,1,1,4,1


Different ways of using `.melt`:

In [ ]:
# df.melt()
# df.melt(id_vars='a')
# df.melt(id_vars='x')
# df.melt(id_vars=['a', 'b'])
# df.melt(value_vars=['x', 'y', 'z'])
# df.melt(id_vars=['a', 'b'], value_vars=['x', 'y'], var_name='foo', value_name='bar')

### Pivot Table Example

In [3]:
import itertools as it

df = pd.DataFrame(it.product('ABC', ['one', 'two', 'three']), columns=['group', 'subgroup'])
df['x'] = np.random.randn(df.shape[0])
df

,group,subgroup,x
0,A,one,-0.769433
1,A,two,0.576746
2,A,three,0.126526
3,B,one,-1.301489
4,B,two,2.207427
5,B,three,0.522742
6,C,one,0.465645
7,C,two,0.724915
8,C,three,1.495827


In [4]:
df.pivot_table(values='x', index='subgroup', columns='group')

group,A,B,C
subgroup,,,
one,-0.769433,-1.301489,0.465645
three,0.126526,0.522742,1.495827
two,0.576746,2.207427,0.724915


## Tidy Data

Tidy Data Characteristics:

- data is tabular, i.e. made up of rows and columns
- there is one value per cell
- each variable is a column
- each observation is a row

General Ideas

- Each variable is a characteristic of an observation
- If the units are the same, maybe they should be in the same column
- If one column has measurements of different units, it should be spread out
- Should you be able to groupby some of the columns? combine them
- Can I pass this data to seaborn?
- Can we ask interesting questions and answer them with a group by? I.e. generally we **don't** want to be taking row or column averages.

For the rest of this lesson, we'll look at data that is **not** tidy.

### One Column with Multiple Variables

In [5]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

,name,pet
0,Sally,dog: max
1,Jane,dog: buddy
2,Billy,cat: grizabella
3,Suzy,hamster: fred


### One Variable in Multiple Columns

In [6]:
np.random.seed(123)

df = pd.DataFrame(
    np.random.uniform(60, 100, (4, 4)),
    columns=['Sally', 'Jane', 'Billy', 'Suzy'],
    index = pd.Index(['spelling', 'math', 'reading', 'nuclear physics'], name='subject')
).round(1).reset_index()
df

,subject,Sally,Jane,Billy,Suzy
0,spelling,87.9,71.4,69.1,82.1
1,math,88.8,76.9,99.2,87.4
2,reading,79.2,75.7,73.7,89.2
3,nuclear physics,77.5,62.4,75.9,89.5


- what is the average spelling grade?
- What is Jane's average grade?

Sometimes it is desirable to "untidy" the data for quick analysis / visualization. E.g. spread subject out to columns, students as rows.

### Multiple vars in 2 columns

- "incorrect melt"

In [7]:
df = pd.read_sql('SELECT * FROM gapminder1', DB_URL)
df.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


### Another gnarly example

In [26]:
df = pd.read_sql('SELECT * FROM gapminder2', DB_URL)
df.head()

,country,life_expect_1955,life_expect_1960,life_expect_1965,life_expect_1970,life_expect_1975,life_expect_1980,life_expect_1985,life_expect_1990,life_expect_1995,...,pop_1960,pop_1965,pop_1970,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005
0,Afghanistan,30.332,31.997,34.020,36.088,38.438,39.854,40.822,41.674,41.763,...,9829450,10997885,12430623,14132019,15112149,13796928,14669339,20881480,23898198,29928987
1,Argentina,64.399,65.142,65.634,67.065,68.481,69.942,70.774,71.868,73.275,...,20616009,22283100,23962313,26081880,28369799,30675059,33022202,35311049,37497728,39537943
2,Aruba,64.381,66.606,68.336,70.941,71.830,74.116,74.494,74.108,73.011,...,57203,59020,59039,59390,60266,64129,66653,67836,69539,71566
3,Australia,70.330,70.930,71.100,71.930,73.490,74.740,76.320,77.560,78.830,...,10361273,11439384,12660160,13771400,14615900,15788300,17022133,18116171,19164620,20090437
4,Austria,67.480,69.540,70.140,70.630,72.170,73.180,74.940,76.040,77.510,...,7047437,7270889,7467086,7578903,7549433,7559776,7722953,8047433,8113413,8184691


In [27]:
df = df.melt(id_vars='country')

In [28]:
df['year'] = df.variable.str[-4:]
df['measure'] = df.variable.str[:-4]
df = df.drop(columns='variable')
df = df.rename(columns={'value':'measurement'})


In [30]:
df[['year', 'country', 'measure', 'measurement']]

,year,country,measure,measurement
0,1955,Afghanistan,life_expect_,3.033200e+01
1,1955,Argentina,life_expect_,6.439900e+01
2,1955,Aruba,life_expect_,6.438100e+01
3,1955,Australia,life_expect_,7.033000e+01
4,1955,Austria,life_expect_,6.748000e+01
...,...,...,...,...
1381,2005,Switzerland,pop_,7.489370e+06
1382,2005,Turkey,pop_,6.966056e+07
1383,2005,United Kingdom,pop_,6.044146e+07
1384,2005,United States,pop_,2.957341e+08


In [9]:
df = df.melt(id_vars='country')
df['year'] = df.variable.str[-4:]
df['measure'] = df.variable.str[:-5]
df = df.drop(columns=['variable'])
df = df.pivot_table('value', ['country', 'year'], 'measure').reset_index()
df.columns.name = ''
df

,country,year,life_expect,pop
0,Afghanistan,1955,30.332,8891209.0
1,Afghanistan,1960,31.997,9829450.0
2,Afghanistan,1965,34.020,10997885.0
3,Afghanistan,1970,36.088,12430623.0
4,Afghanistan,1975,38.438,14132019.0
...,...,...,...,...
688,Venezuela,1985,70.190,16997509.0
689,Venezuela,1990,71.150,19325222.0
690,Venezuela,1995,72.146,21555902.0
691,Venezuela,2000,72.766,23542649.0


### A More Complex Example

This one is the same as the last one in the curriculum.

In [10]:
sales = pd.read_sql('SELECT * FROM sales', DB_URL)
sales

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [11]:
sales_melt = sales.melt(id_vars='Product', var_name='year_and_measure')

year_and_measure_df = sales_melt.year_and_measure.str.split(' ', expand=True)
year_and_measure_df.columns = ['year', 'measure']

sales2 = pd.concat([sales_melt, year_and_measure_df], axis=1).drop(columns='year_and_measure')

sales_tidy = sales2.pivot_table(index=['Product', 'year'], columns='measure', values='value')

sales_tidy.columns.name = ''
sales_tidy.reset_index(inplace=True)
sales_tidy

,Product,year,PPU,Sales
0,A,2016,5,673
1,A,2017,7,231
2,A,2018,9,173
3,B,2016,3,259
4,B,2017,5,748
5,B,2018,8,186
6,C,2016,3,644
7,C,2017,5,863
8,C,2018,5,632
9,D,2016,9,508
